In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from config import covid_api
from config import google_api
import gmaps

# -----------------------
# COVID testing location data
# From https://github.com/codersagainstcovidorg/covid19testing-backend
# -----------------------

testing_df = pd.DataFrame(columns=['Name','Lat','Lon','City','State','Collecting','Evaluating'])
testing_df

base_url = "http://api.findcovidtesting.com/api/v1/location"
response = requests.get(base_url).json()

# -----------------------
# Use this to see how the response is structured:
#response[0]

# -----------------------
# Relevant info seems to be:
#     location_name
#     is_collecting_samples - boolean
#     is_evaluating_symptoms - boolean
#     location_address_locality = city
#     location_address_region = state
#     location_latitude
#     location_longitude


In [2]:
# Iterate through the response and store selected variables in dataframe

for i in range(len(response)):
    if response[i]['location_address_region'] == 'NC':
        testing_df.loc[i,'Name'] = response[i]['location_name']
        testing_df.loc[i,'Lat'] = response[i]['location_latitude']
        testing_df.loc[i,'Lon'] = response[i]['location_longitude']
        testing_df.loc[i,'City'] = response[i]['location_address_locality']
        testing_df.loc[i,'State'] = response[i]['location_address_region']
        testing_df.loc[i,'Collecting'] = response[i]['is_collecting_samples']
        testing_df.loc[i,'Evaluating'] = response[i]['is_evaluating_symptoms']

# Display the dataframe       
testing_df

,Name,Lat,Lon,City,State,Collecting,Evaluating
1,Southside Recreation Center (Guilford County D...,35.9485,-80.0088,High Point,NC,True,True
23,MEDAC Urgent Care -Shipyard,34.1988,-77.8901,Wilmington,NC,True,True
78,Family Medicine-Sunset Avenue,35.7059,-79.8188,Asheboro,NC,True,False
123,AppHealthCare - Alleghany Health Center,36.4924,-81.1459,Sparta,NC,True,True
129,FastMed Urgent Care,35.0086,-80.5616,Monroe,NC,True,True
...,...,...,...,...,...,...,...
9750,City of High Point Parking Deck (Harris Teeter),35.9563,-80.0065,High Point,NC,True,True
9776,New Hanover Regional Medical Center,34.2057,-77.9211,New Hanover,NC,True,False
9791,Macon County Public Health,35.1969,-83.3715,Franklin,NC,True,True
9842,Washington Regional Medical Center,35.8628,-76.7293,Plymouth,NC,True,True


In [3]:
#----------------------
# Open csv of all NC counties
#----------------------

county_path = "data/nc_counties.csv"
county_df = pd.read_csv(county_path)

county_df['nc_county'].replace(', North Carolina','',regex=True,inplace=True)
county_df

,nc_county,county_seat
0,Alamance County,Graham
1,Alexander County,Taylorsville
2,Alleghany County,Sparta
3,Anson County,Wadesboro
4,Ashe County,Jefferson
...,...,...
95,Wayne County,Goldsboro
96,Wilkes County,Wilkesboro
97,Wilson County,Wilson
98,Yadkin County,Yadkinville


In [4]:
# Add latitude and longitude to the dataframe
county_df['Lat'] = ''
county_df['Lon'] = ''

print('--------------------')
print(f'Begin processing cities')
print('--------------------')

for index, row in county_df.iterrows():
    target_city = f'{row[1]}, North Carolina'
    params = {"address": target_city, "key": google_api}
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params=params)
    city_geo = response.json()
    row[2] = city_geo["results"][0]["geometry"]["location"]["lat"]
    row[3] = city_geo["results"][0]["geometry"]["location"]["lng"]
    print(f'Processing: {target_city}')

print('--------------------')
print(f'Processing complete')
print('--------------------')

--------------------
Begin processing cities
--------------------
Processing: Graham, North Carolina
Processing: Taylorsville, North Carolina
Processing: Sparta, North Carolina
Processing: Wadesboro, North Carolina
Processing: Jefferson, North Carolina
Processing: Newland, North Carolina
Processing: Washington, North Carolina
Processing: Windsor, North Carolina
Processing: Elizabethtown, North Carolina
Processing: Bolivia, North Carolina
Processing: Asheville, North Carolina
Processing: Morganton, North Carolina
Processing: Concord, North Carolina
Processing: Lenoir, North Carolina
Processing: Camden, North Carolina
Processing: Beaufort, North Carolina
Processing: Yanceyville, North Carolina
Processing: Newton, North Carolina
Processing: Pittsboro, North Carolina
Processing: Murphy, North Carolina
Processing: Edenton, North Carolina
Processing: Hayesville, North Carolina
Processing: Shelby, North Carolina
Processing: Whiteville, North Carolina
Processing: New Bern, North Carolina
Proce

In [5]:
county_df

,nc_county,county_seat,Lat,Lon
0,Alamance County,Graham,36.069,-79.4006
1,Alexander County,Taylorsville,35.9217,-81.1769
2,Alleghany County,Sparta,36.5054,-81.1209
3,Anson County,Wadesboro,34.9682,-80.0767
4,Ashe County,Jefferson,36.4204,-81.4734
...,...,...,...,...
95,Wayne County,Goldsboro,35.3849,-77.9928
96,Wilkes County,Wilkesboro,36.146,-81.1606
97,Wilson County,Wilson,35.7213,-77.9155
98,Yadkin County,Yadkinville,36.1346,-80.6595


In [6]:
gmaps.configure(api_key=google_api)

# Build the map for testing locations
# Set center and zoom level
fig = gmaps.figure(map_type='HYBRID',center=(35.3,-79.5), zoom_level=7)

testing_locations = testing_df[['Lat', 'Lon']]
testing_info = testing_df['Name'].tolist()
testing_symbols = gmaps.symbol_layer(testing_locations, fill_color='white', stroke_color='white', scale = 2, info_box_content = testing_info)
fig.add_layer(testing_symbols)

# Build the map for county locations
seat_locations = county_df[['Lat', 'Lon']]
seat_info = county_df['nc_county'].tolist()
seat_symbols = gmaps.symbol_layer(seat_locations, fill_color='blue', stroke_color='blue', scale = 3, info_box_content = seat_info)
fig.add_layer(seat_symbols)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# THIS DOESN'T WORK YET

my_phrase = "north carolina hospitals"
target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
next_token = ''

tokens = []

params = {
    "query": my_phrase,
    "key": google_api,
}



response = requests.get(target_url, params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)
hospitals = response.json()

next_token = hospitals['next_page_token']
#hospitals['results'][0]['name']